In [46]:
import pandas as pd
import string
import re
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
import nltk
import csv
from nltk.corpus import stopwords
# nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer 
ps=PorterStemmer()

import ast
import tensorflow as tf
import torch
from transformers import BertTokenizer, BertModel
from transformers import AutoModel, AutoTokenizer
from collections import defaultdict
import json

"""
2 corpora - before and after. split().-> bag of words - remove their names to @user for all of them within the content of the tweet not the person who tweeted. Pull the pre-trained bert vectors - hugging face - interate through the list - get the vectors of each words, dont remove duplicates (if dont find vectors for a word - have an output for those words - see if theres any spelling). we can also lemantize them, try to get hte vectors for those lemna, if still no vector - output, keep them somewhere. 
Dictionary: {Day: {tweet_id: {word}: []}, {word []}, etc.}

Once we have the list of words of dehumanizing language, count/number of words in your corpus: (e.g.: 2% were dehumanizing before, 7% after, frequency for each day and have a time-series analysis)
"""

'\n2 corpora - before and after. split().-> bag of words - remove their names to @user for all of them within the content of the tweet not the person who tweeted. Pull the pre-trained bert vectors - hugging face - interate through the list - get the vectors of each words, dont remove duplicates (if dont find vectors for a word - have an output for those words - see if theres any spelling). we can also lemantize them, try to get hte vectors for those lemna, if still no vector - output, keep them somewhere. \nDictionary: {Day: {tweet_id: {word}: []}, {word []}, etc.}\n\nOnce we have the list of words of dehumanizing language, count/number of words in your corpus: (e.g.: 2% were dehumanizing before, 7% after, frequency for each day and have a time-series analysis)\n'

In [47]:
def remove_stop_words(text: string):
    return ' '.join([item for item in text.split() if item not in stopwords.words('english')])

def remove_punctuations(text: string):
    return "".join([char for char in text if char not in string.punctuation])

def delete_usernames(text: str) -> str:
    pattern = r'@\w+'
    return re.sub(pattern, '@user', text)

def remove_non_words(text: string):
    return re.sub("[^a-zA-Z]",  # Search for all non-letters
                          " ",          # Replace all non-letters with spaces
                          str(text))


# Function to split "After Corpus" into individual sentences and create a new DataFrame
def split_sentences(df, status: str):
    abbreviations = {
        "U.S.": "USA",
        "U.K.": "UK",
        "e.g.": "for example,",
        "i.e.": "such as,",
        "U.N.": "UN",
        "Gov.": "Governor",
        "etc.": "..."
    }

    # Initialize an empty list to store the rows of the new DataFrame
    rows = []

    # Iterate through each row in the original DataFrame
    for _, row in df.iterrows():
        # Extract the relevant data
        subject_id = row['Subject_ID']
        tweet_id = row['Tweet_ID']
        corpus = row[f'{status}_Corpus']

        # Replace abbreviations with placeholders
        for abbr, placeholder in abbreviations.items():
            corpus = corpus.replace(abbr, placeholder)
        
        # Split the corpus into sentences
        sentences = re.split(r'(?<=[.!?])\s+', corpus)

        # Add each sentence as a new row in the list, keeping track of the tweet ID and subject ID
        for sentence in sentences:
            rows.append({'Subject_ID': subject_id, 'Tweet_ID': tweet_id, 'Sentence': sentence})

    # Create a new DataFrame from the list of rows
    new_df = pd.DataFrame(rows)
    return new_df

# def stem_text(text: string):
#     return [ps.stem(word) for word in text]

def get_segments_ids(df):
    segments_ids = []
    for idx, row in df.iterrows():
        # Create a list of repeated IDs for each sentence
        segment_id_list = [idx] * len(row['Indexed_Tokens'])
        segments_ids.append(segment_id_list)
    df["Segments_IDs"] = segments_ids
    return df["Segments_IDs"]

In [48]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [49]:
def process_bert_sentences(sentences_df):
    """
    Process sentences through BERT after padding.
    """
    # Get the maximum length of sequences
    max_len = max(len(seq) for seq in sentences_df['Indexed_Tokens'])
    
    # Pad sequences to the same length
    padded_tokens = []
    padded_segments = []
    
    for tokens, segments in zip(sentences_df['Indexed_Tokens'], sentences_df['Segments_IDs']):
        # Ensure tokens and segments are lists
        tokens = list(tokens)
        segments = list(segments)
        
        # Pad if necessary
        if len(tokens) < max_len:
            tokens = tokens + [0] * (max_len - len(tokens))
        if len(segments) < max_len:
            segments = segments + [0] * (max_len - len(segments))
            
        # Truncate if necessary (BERT typically handles up to 512 tokens)
        tokens = tokens[:512]
        segments = segments[:512]
        
        padded_tokens.append(tokens)
        padded_segments.append(segments)
    
    # Convert to tensors
    tokens_tensor = torch.tensor(padded_tokens, dtype=torch.long)
    segments_tensor = torch.tensor(padded_segments, dtype=torch.long)

    # print(tokens_tensor[:5])
    # print(segments_tensor[:5])
    
    # Create attention mask
    attention_mask = (tokens_tensor != 0).long()
    
    # Initialize model
    model = BertModel.from_pretrained('bert-base-uncased',
                                    output_hidden_states=True,
                                    return_dict=True)
    
    # Evaluation mode
    model.eval()
    
    # # Move to GPU if available
    # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # model = model.to(device)
    # tokens_tensor = tokens_tensor.to(device)
    # segments_tensor = segments_tensor.to(device)
    # attention_mask = attention_mask.to(device)
    
    # Process through BERT
    with torch.no_grad():
        outputs = model(
            input_ids=tokens_tensor,
            token_type_ids=segments_tensor,
            attention_mask=attention_mask
        )
    
    return outputs

In [50]:
#Pre-process Text Data
clean_data_path = "Cleaned Data"
after = pd.read_csv(f"{clean_data_path}/After_NN_Cleaned.csv")
before = pd.read_csv(f"{clean_data_path}/Before_NN_Cleaned.csv")

In [51]:
# Apply the function to split sentences
after_sentences = split_sentences(after, "After")
before_sentences = split_sentences(before, "Before")

# Apply all cleaning functions to the After Corpus column
after_sentences["Sentence"] = (after_sentences["Sentence"]
                             .apply(delete_usernames)
                             .apply(remove_punctuations)
                             .apply(remove_stop_words)
                             .apply(remove_non_words))

before_sentences["Sentence"] = (before_sentences["Sentence"]
                             .apply(delete_usernames)
                             .apply(remove_punctuations)
                             .apply(remove_stop_words)
                             .apply(remove_non_words))

# Apply sentence markers
after_sentences["Sentence"] = ("[CLS] "+ after_sentences["Sentence"] + " [SEP]")
before_sentences["Sentence"] = ("[CLS] "+ before_sentences["Sentence"] + " [SEP]")

# Show the first few rows of the resulting DataFrame
after_sentences.head()

,Subject_ID,Tweet_ID,Sentence
0,74,609,[CLS] hey remember phoned despot bestie Netany...
1,74,609,[CLS] ace job homeslice [SEP]
2,74,609,[CLS] good thing meddling caused serious conse...
3,74,705,[CLS] hey remember Jared Kushner brought peace...
4,74,705,[CLS] neither [SEP]


In [52]:
# Tokenize our sentence with the BERT tokenizer.
after_sentences["Tokens"] = after_sentences["Sentence"].apply(tokenizer.tokenize)
before_sentences["Tokens"] = before_sentences["Sentence"].apply(tokenizer.tokenize)

#Getting Indexed Tokens using BERT tokenzier
after_sentences["Indexed_Tokens"] = after_sentences["Tokens"].apply(lambda x: [tokenizer.convert_tokens_to_ids(token) for token in x])
before_sentences["Indexed_Tokens"] = before_sentences["Tokens"].apply(lambda x: [tokenizer.convert_tokens_to_ids(token) for token in x])

#Getting Segment IDs based on the number of sentences, all tokens in the same sentence get the same ID.
after_sentences["Segments_IDs"] = get_segments_ids(after_sentences)
before_sentences["Segments_IDs"] = get_segments_ids(before_sentences)

before_sentences.head()

,Subject_ID,Tweet_ID,Sentence,Tokens,Indexed_Tokens,Segments_IDs
0,74,353,[CLS] holy fucking shit criticizing Israel ant...,"[[CLS], holy, fucking, shit, criticizing, isra...","[101, 4151, 8239, 4485, 21289, 3956, 3424, 336...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,74,353,[CLS] IS antisemitic putting holocaustdenying ...,"[[CLS], is, anti, ##se, ##mit, ##ic, putting, ...","[101, 2003, 3424, 3366, 22930, 2594, 5128, 115...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,74,353,[CLS] sit fuck shut fuck spare us fucking fake...,"[[CLS], sit, fuck, shut, fuck, spare, us, fuck...","[101, 4133, 6616, 3844, 6616, 8622, 2149, 8239...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]"
3,74,538,[CLS] congratulations literally zero Republica...,"[[CLS], congratulations, literally, zero, repu...","[101, 23156, 6719, 5717, 10643, 2768, 7939, 23...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]"
4,74,538,[CLS] take victory lap spineless fucking cowar...,"[[CLS], take, victory, lap, spine, ##less, fuc...","[101, 2202, 3377, 5001, 8560, 3238, 8239, 1659...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4]"


In [53]:
after_sentences.to_csv("Tokenized Data/After Tokenized.csv", index=False)
before_sentences.to_csv("Tokenized Data/Before Tokenized.csv", index=False)

In [54]:
# Process the sentences
outputs = process_bert_sentences(after_sentences)

# Get hidden states (they're in outputs[2])
hidden_states = outputs[2]

# Print information about the hidden states
print(f"Number of layers: {len(hidden_states)}")
print(f"Shape of hidden states from first layer: {hidden_states[0].shape}")

IndexError: index out of range in self

In [ ]:
#50 vectors

# Convert inputs to PyTorch tensors
before_tokens_tensor = before_sentences["Indexed_Tokens"].apply(torch.tensor)
before_segments_tensors = before_sentences["Segments_IDs"].apply(torch.tensor)
# Create attention mask
attention_mask = (before_tokens_tensor != 0).long()

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers. 
with torch.no_grad():
    outputs = model(
        input_ids=before_tokens_tensor,
        token_type_ids=before_segments_tensors,
        attention_mask=attention_mask
    )
    
    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel

hidden_states = outputs[2]

RuntimeError: Boolean value of Tensor with more than one value is ambiguous